# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
df = pd.read_csv('previsao_de_renda.csv').dropna().drop_duplicates()
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('data_ref', axis=1, inplace=True)
df.drop('id_cliente', axis=1, inplace=True)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12427 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   12427 non-null  object 
 1   posse_de_veiculo       12427 non-null  bool   
 2   posse_de_imovel        12427 non-null  bool   
 3   qtd_filhos             12427 non-null  int64  
 4   tipo_renda             12427 non-null  object 
 5   educacao               12427 non-null  object 
 6   estado_civil           12427 non-null  object 
 7   tipo_residencia        12427 non-null  object 
 8   idade                  12427 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  12427 non-null  float64
 11  renda                  12427 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.1+ MB


1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

##### Parte 1

In [4]:
for col in df.columns:
    if col == 'renda':
        continue
    print(f'{df[col].name}: {df[col].value_counts().index[0]}')

sexo: F
posse_de_veiculo: False
posse_de_imovel: True
qtd_filhos: 0
tipo_renda: Assalariado
educacao: Secundário
estado_civil: Casado
tipo_residencia: Casa
idade: 40
tempo_emprego: 4.216438356164384
qt_pessoas_residencia: 2.0


In [5]:
y_a, X_a = patsy.dmatrices('np.log(renda) ~ '
    'sexo + posse_de_veiculo + '
    'C(posse_de_imovel, Treatment(True)) + '
    'qtd_filhos + '#'C(qtd_filhos) + '
    'tipo_renda + '
    'C(educacao, Treatment("Secundário")) + '
    'estado_civil + '
    'C(tipo_residencia, Treatment("Casa")) + '
    'idade + '
    'tempo_emprego + '
    'qt_pessoas_residencia',#'C(qt_pessoas_residencia, Treatment(2.0))', 
data=df)

reg_a = sm.OLS(y_a, X_a).fit()
reg_a.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Wed, 28 Sep 2022   Prob (F-statistic):               0.00
Time:                        14:56:48   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.6093      0.219     30.214      0.000       6.181       7.038
sexo[T.M]                                                       0.7874      0.015     53.723      0.000       0.759       0.816
posse_de_veiculo[T.True]                                        0.0441      0.014      3.119      0.002       0.016       0.072
C(posse_de_imovel, Treatment(True))[T.False]                   -0.0829      0.014     -5.926      0.000      -0.110      -0.055
tipo_renda[T.Bolsista]                                          0.2209      0.241      0.916      0.360      -0.252       0.694
tipo_renda[T.Empresário]                                        0.1551      0.015     10.387      0.000       0.126       0.184
tipo_renda[T.Pensionista]                                      -0.3087      0.241     -1.280      0.201      -0.782       0.164
tipo_renda[T.Servidor público]                                  0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao, Treatment("Secundário"))[T.Primário]                0.0141      0.072      0.196      0.844      -0.127       0.155
C(educacao, Treatment("Secundário"))[T.Pós graduação]           0.1212      0.142      0.853      0.394      -0.157       0.400
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.1079      0.014      7.763      0.000       0.081       0.135
C(educacao, Treatment("Secundário"))[T.Superior incompleto]    -0.0290      0.032     -0.900      0.368      -0.092       0.034
estado_civil[T.Separado]                                        0.3265      0.111      2.935      0.003       0.108       0.545
estado_civil[T.Solteiro]                                        0.2672      0.109      2.453      0.014       0.054       0.481
estado_civil[T.União]                                          -0.0343      0.025     -1.361      0.174      -0.084       0.015
estado_civil[T.Viúvo]                                           0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia, Treatment("Casa"))[T.Aluguel]                0.0442      0.054      0.815      0.415      -0.062       0.150
C(tipo_residencia, Treatment("Casa"))[T.Com os pais]            0.0184      0.029      0.625      0.532      -0.039       0.076
C(tipo_residencia, Treatment("Casa"))[T.Comunitário]           -0.0754      0.091     -0.829      0.407      -0.254       0.103
C(tipo_residencia, Treatment("Casa"))[T.Estúdio]                0.1107      0.084      1.319      0.187      -0.054       0.275
C(tipo_residencia, Treatment("Casa"))[T.Governamental]         -0.0129      0.039     -0.331      0.741      -0.089       0.064
qtd_filhos 

Há uma quantidade excesssiva de parâmetros. Provavelmente é possível remover alguns.

##### Parte 2

In [6]:
y_b, X_b = patsy.dmatrices('np.log(renda) ~ '
    'sexo + '
    'posse_de_veiculo + '
    'C(posse_de_imovel, Treatment(True)) + '
    'qtd_filhos + '#'C(qtd_filhos) + '
    'tipo_renda + '
    'C(educacao, Treatment("Secundário")) + '
    'estado_civil + '
    #'C(tipo_residencia, Treatment("Casa")) + '
    'idade + '
    'tempo_emprego + '
    'qt_pessoas_residencia', #'C(qt_pessoas_residencia, Treatment(2.0))', 
data=df)

reg_b = sm.OLS(y_b, X_b).fit()
reg_b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     363.0
Date:                Wed, 28 Sep 2022   Prob (F-statistic):               0.00
Time:                        14:56:48   Log-Likelihood:                -13569.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12407   BIC:                         2.733e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.6134      0.219     30.247      0.000       6.185       7.042
sexo[T.M]                                                       0.7893      0.015     53.999      0.000       0.761       0.818
posse_de_veiculo[T.True]                                        0.0442      0.014      3.125      0.002       0.016       0.072
C(posse_de_imovel, Treatment(True))[T.False]                   -0.0819      0.014     -5.966      0.000      -0.109      -0.055
tipo_renda[T.Bolsista]                                          0.2219      0.241      0.920      0.357      -0.251       0.695
tipo_renda[T.Empresário]                                        0.1556      0.015     10.433      0.000       0.126       0.185
tipo_renda[T.Pensionista]                                      -0.3116      0.241     -1.292      0.196      -0.784       0.161
tipo_renda[T.Servidor público]                                  0.0589      0.022      2.653      0.008       0.015       0.102
C(educacao, Treatment("Secundário"))[T.Primário]                0.0097      0.072      0.135      0.893      -0.131       0.150
C(educacao, Treatment("Secundário"))[T.Pós graduação]           0.1200      0.142      0.845      0.398      -0.159       0.399
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.1077      0.014      7.761      0.000       0.080       0.135
C(educacao, Treatment("Secundário"))[T.Superior incompleto]    -0.0288      0.032     -0.893      0.372      -0.092       0.034
estado_civil[T.Separado]                                        0.3272      0.111      2.942      0.003       0.109       0.545
estado_civil[T.Solteiro]                                        0.2683      0.109      2.464      0.014       0.055       0.482
estado_civil[T.União]                                          -0.0329      0.025     -1.310      0.190      -0.082       0.016
estado_civil[T.Viúvo]                                           0.3678      0.116      3.182      0.001       0.141       0.594
qtd_filhos                                                     -0.2655      0.108     -2.460      0.014      -0.477      -0.054
idade                                                           0.0052      0.001      6.579      0.000       0.004       0.007
tempo_emprego                                                   0.0617      0.001     59.485      0.000       0.060       0.064
qt_pessoas_residencia                                           0.2923      0.108      2.713      0.007       0.081       0.504
==============================================================================
Omnibus:                        0.825   Durbin-Watson:      

In [7]:
print('Regressão A:')
print(f'R2 ajustado: {reg_a.rsquared_adj}')
print(f'AIC: {reg_a.aic}')

Regressão A:
R2 ajustado: 0.35622700070043956
AIC: 27185.298825201375


In [8]:
print('Regressão B:')
print(f'R2 ajustado: {reg_b.rsquared_adj}')
print(f'AIC: {reg_b.aic}')

Regressão B:
R2 ajustado: 0.35630063820923785
AIC: 27178.88635985151


Com a remoção da variável tipo_residencia, nota-se uma pequena melhora na relação qualidade-complexidade do modelo.

##### Parte 3

In [15]:
df['empresario'] = df['tipo_renda'] == 'Empresário'
df['servidor_publico'] = df['tipo_renda'] == 'Servidor público'

df['educacao_superior'] = df['educacao'] == 'Superior completo'

df['separado'] = df['estado_civil'] == 'Separado'
df['viuvo'] = df['estado_civil'] == 'Viúvo'
df['solteiro'] = df['estado_civil'] == 'Solteiro'

In [16]:
y_c, X_c = patsy.dmatrices('np.log(renda) ~ '
    'sexo + '
    'posse_de_veiculo + '
    'C(posse_de_imovel, Treatment(True)) + '
    'qtd_filhos + '
    'empresario + '
    'servidor_publico + '
    'educacao_superior + '#'C(educacao, Treatment("Secundário")) + '
    'separado + ' #'estado_civil + '
    'viuvo + '
    'solteiro + '
    'idade + '
    'tempo_emprego + '
    'qt_pessoas_residencia',
data=df)

reg_c = sm.OLS(y_c, X_c).fit()
reg_c.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     530.1
Date:                Wed, 28 Sep 2022   Prob (F-statistic):               0.00
Time:                        15:29:58   Log-Likelihood:                -13572.
No. Observations:               12427   AIC:                         2.717e+04
Df Residuals:                   12413   BIC:                         2.728e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                        6.5987      0.218     30.210      0.000       6.171       7.027
sexo[T.M]                                        0.7897      0.015     54.092      0.000       0.761       0.818
posse_de_veiculo[T.True]                         0.0455      0.014      3.230      0.001       0.018       0.073
C(posse_de_imovel, Treatment(True))[T.False]    -0.0833      0.014     -6.077      0.000      -0.110      -0.056
empresario[T.True]                               0.1550      0.015     10.413      0.000       0.126       0.184
servidor_publico[T.True]                         0.0591      0.022      2.661      0.008       0.016       0.103
educacao_superior[T.True]                        0.1096      0.014      8.078      0.000       0.083       0.136
separado[T.True]                                 0.3313      0.111      2.979      0.003       0.113       0.549
viuvo[T.True]                                    0.3713      0.116      3.213      0.001       0.145       0.598
solteiro[T.True]                                 0.2709      0.109      2.489      0.013       0.058       0.484
qtd_filhos                                      -0.2657      0.108     -2.462      0.014      -0.477      -0.054
idade                                            0.0054      0.001      6.939      0.000       0.004       0.007
tempo_emprego                                    0.0617      0.001     59.468      0.000       0.060       0.064
qt_pessoas_residencia                            0.2931      0.108      2.721      0.007       0.082       0.504
==============================================================================
Omnibus:                        0.824   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.662   Jarque-Bera (JB):                0.808
Skew:                           0.019   Prob(JB):                        0.668
Kurtosis:                       3.010   Cond. No.                     2.13e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
print('Regressão C:')
print(f'R2 ajustado: {reg_c.rsquared_adj}')
print(f'AIC: {reg_c.aic}')

Regressão C:
R2 ajustado: 0.3563099857840597
AIC: 27172.71411733068


A melhora desta vez foi ainda menos significativa.